In [ ]:
#NOTEBOOK TO RANDOMLY EXTRACT THE RUN INFO OF ANY KIND OF EVENT IN ANY OF THE INTERACTION TYPES FOR A SPECIFIC AMOUNT OF PIONS

In [1]:
#INTERACTION TYPE
project_name = 'DIS'

subfolder_name = 'analysisfiles'
folder = f'{project_name}/{subfolder_name}'

In [1]:
import numpy as np
import csv
import math
import matplotlib.pyplot as plt
import decimal
import time
import random
from sklearn.metrics import auc

In [6]:
#automatize the extraction of info of the txt files
def read_file(folder, filename, analysis):
    with open(f"{folder}/{filename}_{analysis}.txt", "r") as file:
        reader = csv.reader(file, delimiter=' ')
        return [row for row in reader]
    
def read_file_old_money(folder, filename):
    with open(f"{folder}/{filename}.txt", "r") as file:
        reader = csv.reader(file, delimiter=' ')
        return [row for row in reader]
    
def read_file2_old_money(folder, filename):
    with open(f"{folder}/{filename}.txt", "r") as file:
        reader = csv.reader(file, delimiter=' ')
        # Skip the first and last element in each row
        return [row[1:-1] for row in reader]
    
    
def filter_runs(energy_range, energy_values, runinfo):
    filtered_runs = []
    for j, energy in enumerate(energy_values):
        if energy_range[0] <= float(energy) < energy_range[1]:
            filtered_runs.append(runinfo[j])
    return filtered_runs

def write_to_file(filename, runs):
    with open(filename, 'w') as file:
        for state in runs:
            file.write(' '.join(map(str, state)) + '\n')
            
def find_event_info(runinfo_list, energies_list, analysis_num, run, subrun, evt, slc):
    for j, event in enumerate(runinfo_list):
        if event[0] == run and event[1] == subrun and event[2] == evt and event[3] == slc:
            print("Event found at index:", j)
            print("Energy information:", energies_list[j])
            print("Vertex Position:", vertex_list[j])
            print("Number of NT pions:", numpint[j])
            print("Number of Reco pions:", numpiReco[j])
            return
    
    print("Event not found.")
    
def find_event_info_old_money(runinfo_list, calenergies_list, cvn_list, pdgpng_list, pdgnt_list, energiesnt_list, runfileevent):
    for j, event in enumerate(runinfo_list):
        if event[0] == runfileevent[0] and event[1] == runfileevent[1] and event[2] == runfileevent[2] and event[3] == runfileevent[3]:
            print("Event: ",runfileevent)
            print("Event found at index:", j)
            print("CVN  info: ", cvn_list[j])
            print("PDGR info: ", pdgpng_list[j])
            print("CalE info: ", calenergies_list[j])
            print("PDGN info: ", pdgnt_list[j])
            print("ENT  info: ", energiesnt_list[j])
            
            return
    
    print("Event not found.")
    
    
def find_index(runfile, runinfo):
    for j, event in enumerate(runfile):
        if event[0] == runinfo[0] and event[1] == runinfo[1] and event[2] == runinfo[2] and event[3] == runinfo[3] and event[4] == runinfo[4]:
            
            return j

In [10]:
start_time = time.time()

Lnumpint_part1 = read_file(folder, 'numpiNT', 'part1')
LnumpiR_part1 = read_file(folder, 'numpiReco', 'part1')
Lnumpint_part2 = read_file(folder, 'numpiNT', 'part2')
LnumpiR_part2 = read_file(folder, 'numpiReco', 'part2')

LenergiesNT_part1 = read_file(folder, 'energiesNT', 'part1')
LenergiesNT_part2 = read_file(folder, 'energiesNT', 'part2')

Lruninfo_part1= read_file(folder, 'runinfo', 'part1')
Lruninfo_part2= read_file(folder, 'runinfo', 'part2')

Lcalenergies = read_file(folder, 'calenergies', 'part1')
Lcvn = read_file(folder, 'CVN', 'part1')

Ltheta_part1 = read_file(folder, 'theta', 'part1')
Ltheta_part2 = read_file(folder, 'theta', 'part2')

Lalpha_part1 = read_file(folder, 'alpha', 'part1')
Lalpha_part2 = read_file(folder, 'alpha', 'part2')

Lnue_part1 = read_file(folder, 'nue', 'part1')
Lnue_part2 = read_file(folder, 'nue', 'part2')

Lvtxpos_part1 = read_file(folder, 'vtxpos', 'part1')
Lvtxpos_part2 = read_file(folder, 'vtxpos', 'part2')

LpdgNTALL=read_file2_old_money(project_name,'pdgntvars_total')
LruninfoALL=read_file_old_money(project_name,'runvars_total')

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 8.29807424545288 seconds


In [3]:
#THIS CODE IS ONLY FOR EXTRACTING THE RUN INFO OF EVENTS WITH ONE NT PION AND ONE RECO (PART1) OR NO RECO (PART2)
start_time = time.time()


Npi_nt = '1'

#Select the analysis type
analysis = "part1" 

#how many events do you want?
evnums = 15
    

if analysis == "part1":
    Npi_Reco = '1'
    Lnumpint = Lnumpint_part1
    LnumpiR = LnumpiR_part1
    LenergiesNT = LenergiesNT_part1
    Lruninfo = Lruninfo_part1
elif analysis == "part2":
    Npi_Reco = '0'
    Lnumpint = Lnumpint_part2
    LnumpiR = LnumpiR_part2
    LenergiesNT = LenergiesNT_part2
    Lruninfo = Lruninfo_part2
else:
    raise ValueError("Invalid analysis value. Must be 'part1' or 'part2'.")

# Define energy ranges and corresponding filenames
energy_ranges = [(0, 0.2), (0.2, 0.5), (0.5, 1), (1, 1.5), (1.5, 2), (2,2.5)]
filenames = [f'{folder}/evd/{Npi_nt}pion/run_E{i}_{Npi_nt}pion_{analysis}.txt' for i in range(len(energy_ranges))]

# Initialize a list to store filtered runs for each energy range
runs = [[] for _ in range(len(energy_ranges))] 

for j, event in enumerate(Lnumpint):
    if event[0] == Npi_nt and LnumpiR[j][0] == Npi_Reco:
        energy = float(LenergiesNT[j][0])
        for i, (start, end) in enumerate(energy_ranges):
            if start <= energy < end:
                runs[i].append(Lruninfo[j])

# Select a number of random events from each energy range
runs_selected = [[random.choice(run_list) for _ in range(evnums)] for run_list in runs]

# Write selected runs to files
for filename, runs_list in zip(filenames, runs_selected):
    write_to_file(filename, runs_list)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")


NameError: name 'Lnumpint_part1' is not defined

In [26]:
# Ask for a event with the runinfo and all the info about it will be displayed
input_str = input("Enter the run number, subrun number, event number, slice number, and analysis part number separated by spaces: ")
run, subrun, evt, slc, analysis_num = input_str.split()

# Convert analysis_num to integer
analysis_num = int(analysis_num)

# Determine the analysis part based on user input
if analysis_num == 1:
    runinfo_list = Lruninfo_part1
    energies_list = LenergiesNT_part1
    vertex_list = Lvtxpos_part1
    numpint = Lnumpint_part1
    numpiReco = LnumpiR_part1
elif analysis_num == 2:
    runinfo_list = Lruninfo_part2
    energies_list = LenergiesNT_part2
    vertex_list = Lvtxpos_part2
    numpint = Lnumpint_part2
    numpiReco = LnumpiR_part1
else:
    print("Invalid analysis number.")

# Call the function
if analysis_num in [1, 2]:
    find_event_info(runinfo_list, energies_list, analysis_num, run, subrun, evt, slc)

Enter the run number, subrun number, event number, slice number, and analysis part number separated by spaces: 11979 1 3 145 2
Event found at index: 11493
Energy information: ['0.197157']
Vertex Position: ['-147.058', '72.726', '363.73']
Number of NT pions: ['1']
Number of Reco pions: ['1']


In [31]:
# check whats happening with the low energy pions theta angles
start_time = time.time()

#this is for part 1 

#how many events do you want?
evnums = 10

runs = []

calenergies = Lcalenergies
Ltheta=Ltheta_part1
Lruninfo = Lruninfo_part1


# Define energy ranges and corresponding filenames
# energy_ranges = [(0, 0.2), (0.2, 0.5), (0.5, 1), (1, 1.5), (1.5, 2), (2,2.5)]
# filenames = [f'{folder}/evd/{Npi_nt}pion/run_E{i}_{Npi_nt}pion_{analysis}.txt' for i in range(len(energy_ranges))]

# Initialize a list to store filtered runs for each energy range
# runs = [[] for _ in range(len(energy_ranges))] 

for i, event in enumerate(calenergies):
    if (float(event[0])-0.139) < 0.2:
        if float(Ltheta[i][0])*(180/math.pi) > 80 and float(Ltheta[i][0])*(180/math.pi) < 120:
            runs.append(Lruninfo[i])
        
# Select 10 random rows from the list
selected_runs = random.sample(runs, 10)

with open(f'{folder}/evd/check_low_pions_{project_name}_part1.txt', 'w') as file:
    for state in selected_runs:
        file.write(' '.join(map(str, state)) + '\n')
        
        
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")


Elapsed time: 0.16673493385314941 seconds


In [32]:
# check whats happening with the low energy pions theta angles
start_time = time.time()

#and this for part 2

#how many events do you want?
evnums = 10

runs = []

energies = LenergiesNT_part2
Ltheta=Ltheta_part2
Lruninfo = Lruninfo_part2


# Define energy ranges and corresponding filenames
# energy_ranges = [(0, 0.2), (0.2, 0.5), (0.5, 1), (1, 1.5), (1.5, 2), (2,2.5)]
# filenames = [f'{folder}/evd/{Npi_nt}pion/run_E{i}_{Npi_nt}pion_{analysis}.txt' for i in range(len(energy_ranges))]

# Initialize a list to store filtered runs for each energy range
# runs = [[] for _ in range(len(energy_ranges))] 

for i, event in enumerate(energies):
    if (float(event[0])-0.139) < 0.2:
        if float(Ltheta[i][0])*(180/math.pi) > 80 and float(Ltheta[i][0])*(180/math.pi) < 120:
            runs.append(Lruninfo[i])
        
# Select 10 random rows from the list
selected_runs = random.sample(runs, 10)

with open(f'{folder}/evd/check_low_pions_{project_name}_part2.txt', 'w') as file:
    for state in selected_runs:
        file.write(' '.join(map(str, state)) + '\n')
        
        
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 0.335284948348999 seconds


In [11]:
# lets extract some events with multiple pions


start_time = time.time()


evnums = 20

runs = []

for i, event in enumerate(LpdgNTALL):
    npions=0
    for element in event:
        if element == '211' or element == '-211':
            npions=npions+1
    if npions>3:
        runs.append(LruninfoALL[i])

        
# Select 10 random rows from the list
selected_runs = random.sample(runs, evnums)

with open(f'{folder}/evd/multi_pion_events_{project_name}.txt', 'w') as file:
    for state in selected_runs:
        file.write(' '.join(map(str, state)) + '\n')
        
        
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 0.39960312843322754 seconds


In [ ]:
#IN THIS, I CAN ASK FOR EVENTS WITH SPECIFIC NUMBER OF NT PIONS JUST FOR EXAMPLE PURPOSES.
start_time = time.time()


Npi_nt = '1'

#DO YOU WANT SPECIFIC NUMBER OF RECO PIONS, UNCOMMENT HERE AND BACK IN ***
#Npi_Reco = '1'

#Select the analysis type
analysis = "part1" 

#how many events do you want?
evnums = 15
    

if analysis == "part1":
    Lnumpint = Lnumpint_part1
    LnumpiR = LnumpiR_part1
    LenergiesNT = LenergiesNT_part1
    Lruninfo = Lruninfo_part1
elif analysis == "part2":
    Lnumpint = Lnumpint_part2
    LnumpiR = LnumpiR_part2
    LenergiesNT = LenergiesNT_part2
    Lruninfo = Lruninfo_part2
else:
    raise ValueError("Invalid analysis value. Must be 'part1' or 'part2'.")

# Define energy ranges and corresponding filenames
energy_ranges = [(0, 0.2), (0.2, 0.5), (0.5, 1), (1, 1.5), (1.5, 2), (2,2.5)]
filenames = [f'{folder}/evd/{Npi_nt}pion/run_E{i}_{Npi_nt}pion_{analysis}.txt' for i in range(len(energy_ranges))]

# Initialize a list to store filtered runs for each energy range
runs = [[] for _ in range(len(energy_ranges))] 

for j, event in enumerate(Lnumpint):
    if event[0] == Npi_nt: # and LnumpiR[j][0] == Npi_Reco:                             #***
        energy = float(LenergiesNT[j][0])
        for i, (start, end) in enumerate(energy_ranges):
            if start <= energy < end:
                runs[i].append(Lruninfo[j])

# Select a number of random events from each energy range
runs_selected = [[random.choice(run_list) for _ in range(evnums)] for run_list in runs]

# Write selected runs to files
for filename, runs_list in zip(filenames, runs_selected):
    write_to_file(filename, runs_list)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

In [47]:
#here, i want to check ALL information, specially cvn values, of the current datasets in my pc
#this is a special code block, with slightly different work flow.

#write folder of interest:

folder='DIS'

#this will load all relevant lists
Lruninfo = read_file_old_money(folder, 'runvars_total')
Lpdgsnt = read_file_old_money(folder, 'pdgntvars_total')
Lpdgspng = read_file_old_money(folder, 'pdgpngvars_total')
Lcvn = read_file_old_money(folder, 'cvnvars_total')
Lenergiesnt = read_file_old_money(folder, 'energiesALLvarsNT_total')
Lcalenergies = read_file_old_money(folder, 'calenergiesvars_total')


In [13]:
#now select the run file:

energy='E0'
part='part1'

runfile=read_file_old_money(folder,f'analysisfiles/evd/1pion/run_{energy}_1pion_{part}')

#search each row into the big run file:

for event in runfile:
    find_event_info_old_money(Lruninfo, Lcalenergies, Lcvn, Lpdgspng, Lpdgsnt, Lenergiesnt, event)
    

Event:  ['12060', '5', '13', '405', '2']
Event found at index: 167266
CVN  info:  ['-5', '0.217205', '0.0167229', '0.509033', '']
PDGR info:  ['-1000', '13', '2212', '211', '']
CalE info:  ['-5', '0.386447', '0.328225', '0.0653232', '']
PDGN info:  ['-1000', '13', '2112', '211', '']
ENT  info:  ['-10', '0.899907', '1.40447', '0.194708', '']
Event:  ['13229', '14', '81', '444', '5']
Event found at index: 139379
CVN  info:  ['-5', '0.197796', '0.989537', '0.879852', '0.529495', '']
PDGR info:  ['-1000', '13', '321', '321', '211', '']
CalE info:  ['-5', '0.517979', '0.460746', '0.237999', '0.0626166', '']
PDGN info:  ['-1000', '13', '3122', '321', '211', '']
ENT  info:  ['-10', '0.541552', '1.46757', '0.996638', '0.171893', '']
Event:  ['11572', '8', '60', '395', '6']
Event found at index: 394545
CVN  info:  ['-5', '0.645877', '0.0225044', '0.534268', '']
PDGR info:  ['-1000', '13', '-211', '211', '']
CalE info:  ['-5', '0.44977', '0.0852459', '0.0698273', '']
PDGN info:  ['-1000', '13', 

In [3]:
folder='Res'

#this will load all relevant lists
Lruninfo = read_file_old_money(folder, 'runvars_total')
Lpdgsnt = read_file2_old_money(folder, 'pdgntvars_total')
Lpdgspng = read_file2_old_money(folder, 'pdgpngvars_total')
Lcvn = read_file2_old_money(folder, 'cvnvars_total')
Lenergiesnt = read_file2_old_money(folder, 'energiesALLvarsNT_total')
Lcalenergies = read_file2_old_money(folder, 'calenergiesvars_total')
Llength = read_file2_old_money(folder, 'pronglengthvars_total')
Lhits = read_file2_old_money(folder, 'pronghitshvars_total')

In [108]:
print(len(Lpdgspng))

429839


In [7]:
#New second adaptation of the searching code pbut for permanently saving data and then searching through.


eflag=['E0','E1','E2','E3','E4','E5']
pflag=['part1','part2']

fullruninfo={
    'part1_E0': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part1_E1': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part1_E2': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part1_E3': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part1_E4': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part1_E5': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part2_E0': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part2_E1': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part2_E2': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part2_E3': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part2_E4': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
    'part2_E5': {'runinfo': [], 'cvn': [], 'pdgpng': [], 'pdgnt': [], 'calenergies': [], 'energiesnt': [], 'hits': [], 'length': []},
}

#fill the runinfo entries
for part in pflag:
    for energy in eflag:
        runfile=read_file_old_money(folder,f'analysisfiles/evd/1pion/run_{energy}_1pion_{part}')

        for event in runfile:
            fullruninfo[f'{part}_{energy}']['runinfo'].append(event)
            
#then search each runinfo and match it with the corresponding variable info
for part in pflag:
    for energy in eflag:
        for i in range(len(fullruninfo[f'{part}_{energy}']['runinfo'])):
            index=find_index(Lruninfo,fullruninfo[f'{part}_{energy}']['runinfo'][i])
            fullruninfo[f'{part}_{energy}']['cvn'].append(Lcvn[index])
            fullruninfo[f'{part}_{energy}']['pdgpng'].append(Lpdgspng[index])
            fullruninfo[f'{part}_{energy}']['pdgnt'].append(Lpdgsnt[index])
            fullruninfo[f'{part}_{energy}']['energiesnt'].append(Lenergiesnt[index])
            fullruninfo[f'{part}_{energy}']['calenergies'].append(Lcalenergies[index])
            fullruninfo[f'{part}_{energy}']['length'].append(Llength[index])
            fullruninfo[f'{part}_{energy}']['hits'].append(Lhits[index])
        
print('ito')

ito


In [132]:
print(fullruninfo['part1_E1']['energiesnt'][4])
index=find_index(Lruninfo,fullruninfo['part1_E1']['runinfo'][4])
print(index)
# index=39286
print(Lenergiesnt[index])


['0.819899', '1.91877', '0.346509']
39286
['0.819899', '1.91877', '0.346509']


In [8]:
# properly check the info of the event of interest:
# Ask for a event with the runinfo and all the info about it will be displayed
input_str = input("Enter the run number, subrun number, event number, slice number separated by spaces: ")
run, subrun, eventn = input_str.split()

for part in pflag:
    for energy in eflag:
        for i, event in enumerate(fullruninfo[f'{part}_{energy}']['runinfo']):
            if event[0] == run and event[1] == subrun and event[3] == eventn:
                print(event)
                print(fullruninfo[f'{part}_{energy}']['pdgpng'][i])
                print(fullruninfo[f'{part}_{energy}']['cvn'][i])
                print(fullruninfo[f'{part}_{energy}']['calenergies'][i])
                print(fullruninfo[f'{part}_{energy}']['hits'][i])
                print(fullruninfo[f'{part}_{energy}']['length'][i])
                print(fullruninfo[f'{part}_{energy}']['pdgnt'][i])
                print(fullruninfo[f'{part}_{energy}']['energiesnt'][i])
                print(part)
                print(energy)
                print(i)
                
                

Enter the run number, subrun number, event number, slice number separated by spaces: 13154 3 188
['13154', '3', '22', '188', '7']
['13', '13', '211', '2212']
['0.704894', '0.67811', '0.0601321', '0.0885384']
['1.27834', '1.126', '0.677812', '0.315832']
['62', '66', '22', '12']
['363.814', '381.154', '228.41', '63.4227']
['13', '211', '2212', '2212']
['9.21471', '2.08344', '1.24794', '3.01793']
part1
E5
10


In [69]:
#search by pdg and cvn region

myinterestpdg=2212

for part in pflag:
    for energy in eflag:
        for i, event in enumerate(fullruninfo[f'{part}_{energy}']['pdgpng']):
            for j, element in enumerate(fullruninfo[f'{part}_{energy}']['pdgpng'][i]):
                if float(element) < 3000:
                    if int(element) == myinterestpdg:
                        if float(fullruninfo[f'{part}_{energy}']['cvn'][i][j]) < 0.5 : #PUT HERE THE CVN
                            cvnss=fullruninfo[f'{part}_{energy}']['cvn'][i][j]
                            print(f'encontrado en {part}, {energy}, una partícula {element} con cvn {cvnss}, at line {i}')
                            break

encontrado en part1, E0, una partícula 2212 con cvn 0.0167229, at line 0
encontrado en part1, E0, una partícula 2212 con cvn 0.157593, at line 3
encontrado en part1, E0, una partícula 2212 con cvn 5.52362e-05, at line 4
encontrado en part1, E0, una partícula 2212 con cvn 0.151359, at line 11
encontrado en part1, E1, una partícula 2212 con cvn 0.245893, at line 2
encontrado en part1, E1, una partícula 2212 con cvn 0.0616962, at line 4
encontrado en part1, E1, una partícula 2212 con cvn 0.026893, at line 11
encontrado en part1, E2, una partícula 2212 con cvn 0.16134, at line 3
encontrado en part1, E2, una partícula 2212 con cvn 0.26377, at line 5
encontrado en part1, E2, una partícula 2212 con cvn 0.498799, at line 8
encontrado en part1, E2, una partícula 2212 con cvn 0.00394686, at line 9
encontrado en part1, E2, una partícula 2212 con cvn 0.0380111, at line 10
encontrado en part1, E2, una partícula 2212 con cvn 0.120529, at line 12
encontrado en part1, E2, una partícula 2212 con cvn 0.

In [70]:
print(fullruninfo['part1_E0']['cvn'][0])
print(fullruninfo['part1_E0']['pdgpng'][0])

['0.217205', '0.0167229', '0.509033']
['13', '2212', '211']


In [45]:
folder='DIS'
Lruninfo = read_file_old_money(folder, 'runvars_total')
print(find_index(Lruninfo,fullruninfo['part1_E4']['runinfo'][0]))

146719


In [35]:
print(fullruninfo['part1_E4']['runinfo'][0])

['11428', '0', '32', '451', '7']


In [27]:
print(fullruninfo[f'{pflag[0]}_{eflag[0]}']['runinfo'])

[['12060', '5', '13', '405', '2'], ['13229', '14', '81', '444', '5'], ['11572', '8', '60', '395', '6'], ['10993', '2', '28', '716', '3'], ['10935', '21', '37', '311', '3'], ['13470', '5', '58', '21', '1'], ['11392', '0', '11', '364', '1'], ['10970', '10', '30', '574', '1'], ['10919', '15', '40', '210', '3'], ['12074', '20', '33', '412', '6'], ['10925', '16', '25', '438', '2'], ['10838', '19', '4', '569', '2'], ['10777', '21', '19', '816', '4'], ['12019', '15', '45', '14', '1'], ['13501', '15', '6', '152', '1']]
